# sKCSD tutorial
In this tutorial we will cover three topics: data format for sKCSD estimation, sKCSD method and source visualization.  
3 broad topics, we start with idea regular electrode configuration, where we place a known test source to compute the potentials at the electode locations and then use these potentials to perfor the kcsd estimation (Basic features). In the second part of the tutorial, we explore the case of noisy electrodes and test the robustness of the method (Noisy electrodes). In the final part we look at how the errors in the estimation depend of the sources and the electrode configuration by testing broken electrodes (Broken electrodes).

In [5]:
from kcsd import sKCSDcell, sKCSD, sample_data_path
from kcsd import utility_functions as utils
from kcsd.utility_functions import LoadData
import os

In [7]:
data_fname = os.path.join(sample_data_path, "gang_7x7_200")


In [ ]:
data = LoadData(fname)